## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-10-03-52-33 +0000,nyt,Here is the latest.,https://www.nytimes.com/live/2025/10/09/us/tru...
1,2025-10-10-03-42-37 +0000,nypost,Florida man stole $7K in scratch-off lottery t...,https://nypost.com/2025/10/09/us-news/florida-...
2,2025-10-10-03-42-21 +0000,startribune,Takeaways: Ryan Hartman scores twice as Wild b...,https://www.startribune.com/minnesota-wild-bea...
3,2025-10-10-03-22-00 +0000,wsj,"As tensions rise on Capitol Hill, Republican a...",https://www.wsj.com/politics/policy/government...
4,2025-10-10-03-15-22 +0000,nyt,Powerful Earthquake Strikes Southern Philippines,https://www.nytimes.com/2025/10/09/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
144,trump,48
77,gaza,36
79,deal,36
105,new,18
94,israel,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
120,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...,183
164,2025-10-09-19-02-29 +0000,nypost,Rep. Andy Barr latest to nominate Trump for No...,https://nypost.com/2025/10/09/us-news/rep-andy...,161
113,2025-10-09-21-17-33 +0000,nypost,Zohran Mamdani refuses to credit Trump over Ga...,https://nypost.com/2025/10/09/us-news/zohran-m...,155
93,2025-10-09-22-13-20 +0000,nypost,Obama called out for omitting Trump in post ap...,https://nypost.com/2025/10/09/us-news/obama-ap...,148
274,2025-10-09-11-51-45 +0000,nypost,Gazans join Israelis in chanting Trump’s name ...,https://nypost.com/2025/10/09/us-news/gazans-j...,144


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
120,183,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...
76,80,2025-10-09-22-51-00 +0000,wsj,The Justice Department has secured an indictme...,https://www.wsj.com/politics/justice-departmen...
242,58,2025-10-09-14-10-27 +0000,nypost,Hamas praises Trump for brokering hard-fought ...,https://nypost.com/2025/10/09/world-news/hamas...
25,52,2025-10-10-01-15-44 +0000,nyt,Federal Judge Temporarily Blocks Trump’s Natio...,https://www.nytimes.com/2025/10/09/us/judge-na...
288,48,2025-10-09-10-39-00 +0000,wsj,China introduced new controls on rare-earth ex...,https://www.wsj.com/economy/trade/china-impose...
155,47,2025-10-09-19-35-40 +0000,nypost,Chuck Schumer ripped for claiming shutdown is ...,https://nypost.com/2025/10/09/us-news/chuck-sc...
224,35,2025-10-09-15-18-12 +0000,latimes,Older men across L.A. County thought they had ...,https://www.latimes.com/california/story/2025-...
173,35,2025-10-09-18-13-34 +0000,cbc,Trump wants to be 5th U.S. president to win th...,https://www.cbc.ca/news/world/trump-nobel-peac...
53,34,2025-10-09-23-54-00 +0000,wsj,The Bureau of Labor Statistics will bring some...,https://www.wsj.com/politics/policy/federal-go...
160,31,2025-10-09-19-23-47 +0000,nyt,Texas High Court Halts Execution of Robert Rob...,https://www.nytimes.com/2025/10/09/us/texas-sh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
